## Import Packages

In [1]:
import csv
import time
from openai import OpenAI
from os import environ
from dotenv import load_dotenv
import pandas as pd

## API Key and Data

In [2]:
load_dotenv()
client = OpenAI(api_key=environ.get("OPENAI_API_KEY"))

data = pd.read_excel('chat_prompts.xlsx')
data = data.fillna('')

tests = [
    "usmle_1_q"
]

test_answers = [
    "usmle_1_a"
]

## Prompt ChatGPT

In [4]:
for item, test in enumerate(tests):
    for idx, row in enumerate(data.itertuples()):
        if getattr(row, "ne_match") != "CCC" and getattr(row, "ex_match") != "CCC":
            continue
        with open("results_sent_drop.csv", mode="a", encoding="UTF8") as file:
            writer = csv.writer(file)
            csv_out = []
            csv_out.append(idx)

            prompt = getattr(row, test)

            # Remove multiple choice options, making question open-ended
            prompt = prompt.split("(A)")
            prompt = prompt[0]
            prompt = prompt.replace("which of the following", "what")
            prompt = prompt.replace("Which of the following", "What")

            # Split question into sentences
            prompt_split = prompt.split(". ")
            for id, sentence in enumerate(prompt_split):
                if (id + 2) >= len(prompt_split) and "?" in sentence:
                    break
                else:
                    temp_prompt = prompt_split
                    temp_prompt = temp_prompt[:id] + temp_prompt[id+1:]

                    sentence_drop_prompt = ". ".join(temp_prompt)

                    question = f"Including an explanation, answer the following question: {sentence_drop_prompt}"
                    copmpletion = client.chat.completions.create(
                        model="gpt-4",
                        messages=[{"role": "system", "content": question}],
                        max_tokens=2048,
                        n=1,
                        stop=None,
                        temperature=0,
                        top_p=1,
                        frequency_penalty=0,
                        presence_penalty=0
                    )

                    gpt_ans = copmpletion.choices[0].message.content.strip()
                    gpt_ans = " ".join(gpt_ans.splitlines())
                    data.at[idx, f"{test_answers[item]}_{id}"] = gpt_ans
                    csv_out.append(gpt_ans)
                    time.sleep(4)

            writer.writerow(csv_out)
            print(f"Finished question {idx} for {test}")

    data.to_excel(f"output_results_sent_drop.xlsx") 

Finished question 0 for usmle_1_q
Finished question 2 for usmle_1_q
Finished question 3 for usmle_1_q
Finished question 4 for usmle_1_q
Finished question 5 for usmle_1_q
Finished question 7 for usmle_1_q
Finished question 11 for usmle_1_q
Finished question 14 for usmle_1_q
Finished question 16 for usmle_1_q
Finished question 17 for usmle_1_q
Finished question 19 for usmle_1_q
Finished question 22 for usmle_1_q
Finished question 24 for usmle_1_q
Finished question 25 for usmle_1_q
Finished question 27 for usmle_1_q
Finished question 30 for usmle_1_q
Finished question 34 for usmle_1_q
Finished question 35 for usmle_1_q
Finished question 36 for usmle_1_q
Finished question 38 for usmle_1_q
Finished question 44 for usmle_1_q
Finished question 45 for usmle_1_q
Finished question 47 for usmle_1_q
Finished question 50 for usmle_1_q
Finished question 53 for usmle_1_q
Finished question 54 for usmle_1_q
Finished question 55 for usmle_1_q
Finished question 57 for usmle_1_q
Finished question 65 for u